In [5]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy as sp
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle

In [6]:
holdout = pd.read_csv('kc_house_data_test_features.csv', index_col=0)
holdout.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [7]:
with open('model.pickle', 'rb') as f:
    model = pickle.load(f)

In [8]:
with open('scaler.pickle', 'rb') as f:
    scaler = pickle.load(f)

In [10]:
holdout = holdout.drop(['grade','sqft_above','bathrooms','sqft_living15', 'id'], axis=1)
holdout.head()

,date,bedrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_lot15
0,20140827T000000,4,2270,11500,1.0,0,0,3,730,1967,0,98034,47.7089,-122.241,10918
1,20150218T000000,4,2270,11500,1.0,0,0,3,730,1967,0,98034,47.7089,-122.241,10918
2,20141107T000000,3,1470,1779,2.0,0,0,3,310,2005,0,98029,47.5472,-121.998,1576
3,20141203T000000,3,1280,16200,1.0,0,0,3,250,1976,0,98077,47.7427,-122.071,10565
4,20150115T000000,4,2830,8126,2.0,0,0,3,0,2005,0,98059,47.4863,-122.140,7916


In [11]:
holdout['date'] = holdout['date'].str.slice(0,8,1)
holdout['year_sold'] = holdout['date'].str.slice(0,4,1)
holdout['month_sold'] = holdout['date'].str.slice(4,6,1)
holdout['day_sold'] = holdout['date'].str.slice(6,8,1)

In [12]:
holdout['year_sold'] = holdout['year_sold'].astype('int64')
holdout['month_sold'] = holdout['month_sold'].astype('int64')
holdout['day_sold'] = holdout['day_sold'].astype('int64')

In [18]:
holdout['yr_renovated'] = holdout['yr_renovated'].replace(0,holdout['yr_built'])
holdout['years_since_renovation'] = (2020-holdout['yr_renovated'])
holdout.head()

,bedrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_lot15,year_sold,month_sold,day_sold,years_since_renovation
0,4,2270,11500,1.0,0,0,3,730,1967,1967,98034,47.7089,-122.241,10918,2014,8,27,53
1,4,2270,11500,1.0,0,0,3,730,1967,1967,98034,47.7089,-122.241,10918,2015,2,18,53
2,3,1470,1779,2.0,0,0,3,310,2005,2005,98029,47.5472,-121.998,1576,2014,11,7,15
3,3,1280,16200,1.0,0,0,3,250,1976,1976,98077,47.7427,-122.071,10565,2014,12,3,44
4,4,2830,8126,2.0,0,0,3,0,2005,2005,98059,47.4863,-122.140,7916,2015,1,15,15


In [17]:
holdout = holdout.drop(columns='date')

In [22]:
month_sold_dummies = pd.get_dummies(holdout['month_sold'],prefix='month',drop_first=True)
water_dummies = pd.get_dummies(holdout['waterfront'], prefix='waterfront',drop_first=True)
floors_dummies = pd.get_dummies(holdout['floors'], prefix='floors',drop_first=True)
view_dummies = pd.get_dummies(holdout['view'], prefix='view',drop_first=True)
cond_dummies = pd.get_dummies(holdout['condition'],prefix='cond',drop_first=True)
zip_dummies = pd.get_dummies(holdout['zipcode'],prefix='zip',drop_first=True)
bed_dummies = pd.get_dummies(holdout['bedrooms'],prefix='bed',drop_first=True)
yr_built_dummies = pd.get_dummies(holdout['yr_built'],prefix='yrbuilt',drop_first=True)
renovated_dummies = pd.get_dummies(holdout['years_since_renovation'],prefix='renovated',drop_first=True)
dummie_data = pd.concat([month_sold_dummies, water_dummies,floors_dummies,view_dummies,cond_dummies,zip_dummies,bed_dummies,yr_built_dummies,renovated_dummies],axis=1)
dummie_data.head()

,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,waterfront_1,floors_1.5,floors_2.0,floors_2.5,floors_3.0,floors_3.5,view_1,view_2,view_3,view_4,cond_2,cond_3,cond_4,cond_5,zip_98002,zip_98003,zip_98004,zip_98005,zip_98006,zip_98007,zip_98008,zip_98010,zip_98011,zip_98014,zip_98019,zip_98022,zip_98023,zip_98024,zip_98027,zip_98028,zip_98029,zip_98030,zip_98031,zip_98032,zip_98033,zip_98034,zip_98038,zip_98039,zip_98040,zip_98042,zip_98045,zip_98052,zip_98053,zip_98055,zip_98056,zip_98058,zip_98059,zip_98065,zip_98070,zip_98072,zip_98074,zip_98075,zip_98077,zip_98092,zip_98102,zip_98103,zip_98105,zip_98106,zip_98107,zip_98108,zip_98109,zip_98112,zip_98115,zip_98116,zip_98117,zip_98118,zip_98119,zip_98122,zip_98125,zip_98126,zip_98133,zip_98136,zip_98144,zip_98146,zip_98148,zip_98155,zip_98166,zip_98168,zip_98177,zip_98178,zip_98188,zip_98198,zip_98199,bed_1,bed_2,bed_3,bed_4,bed_5,bed_6,bed_7,bed_8,bed_9,bed_10,yrbuilt_1901,yrbuilt_1902,yrbuilt_1903,yrbuilt_1904,yrbuilt_1905,yrbuilt_1906,yrbuilt_1907,yrbuilt_1908,yrbuilt_1909,yrbuilt_1910,yrbuilt_1911,yrbuilt_1912,yrbuilt_1913,yrbuilt_1914,yrbuilt_1915,yrbuilt_1916,yrbuilt_1917,yrbuilt_1918,yrbuilt_1919,yrbuilt_1920,yrbuilt_1921,yrbuilt_1922,yrbuilt_1923,yrbuilt_1924,yrbuilt_1925,yrbuilt_1926,yrbuilt_1927,yrbuilt_1928,yrbuilt_1929,yrbuilt_1930,yrbuilt_1931,yrbuilt_1932,yrbuilt_1933,yrbuilt_1934,yrbuilt_1935,yrbuilt_1936,yrbuilt_1937,yrbuilt_1938,yrbuilt_1939,yrbuilt_1940,yrbuilt_1941,yrbuilt_1942,yrbuilt_1943,yrbuilt_1944,yrbuilt_1945,yrbuilt_1946,yrbuilt_1947,yrbuilt_1948,yrbuilt_1949,yrbuilt_1950,yrbuilt_1951,yrbuilt_1952,yrbuilt_1953,yrbuilt_1954,yrbuilt_1955,yrbuilt_1956,yrbuilt_1957,yrbuilt_1958,yrbuilt_1959,yrbuilt_1960,yrbuilt_1961,yrbuilt_1962,yrbuilt_1963,yrbuilt_1964,yrbuilt_1965,yrbuilt_1966,yrbuilt_1967,yrbuilt_1968,yrbuilt_1969,yrbuilt_1970,yrbuilt_1971,yrbuilt_1972,yrbuilt_1973,yrbuilt_1974,yrbuilt_1975,yrbuilt_1976,yrbuilt_1977,yrbuilt_1978,yrbuilt_1979,yrbuilt_1980,yrbuilt_1981,yrbuilt_1982,yrbuilt_1983,yrbuilt_1984,yrbuilt_1985,yrbuilt_1986,yrbuilt_1987,yrbuilt_1988,yrbuilt_1989,yrbuilt_1990,yrbuilt_1991,yrbuilt_1992,yrbuilt_1993,yrbuilt_1994,yrbuilt_1995,yrbuilt_1996,yrbuilt_1997,yrbuilt_1998,yrbuilt_1999,yrbuilt_2000,yrbuilt_2001,yrbuilt_2002,yrbuilt_2003,yrbuilt_2004,yrbuilt_2005,yrbuilt_2006,yrbuilt_2007,yrbuilt_2008,yrbuilt_2009,yrbuilt_2010,yrbuilt_2011,yrbuilt_2012,yrbuilt_2013,yrbuilt_2014,yrbuilt_2015,renovated_6,renovated_7,renovated_8,renovated_9,renovated_10,renovated_11,renovated_12,renovated_13,renovated_14,renovated_15,renovated_16,renovated_17,renovated_18,renovated_19,renovated_20,renovated_21,renovated_22,renovated_23,renovated_24,renovated_25,renovated_26,renovated_27,renovated_28,renovated_29,renovated_30,renovated_31,renovated_32,renovated_33,renovated_34,renovated_35,renovated_36,renovated_37,renovated_38,renovated_39,renovated_40,renovated_41,renovated_42,renovated_43,renovated_44,renovated_45,renovated_46,renovated_47,renovated_48,renovated_49,renovated_50,renovated_51,renovated_52,renovated_53,renovated_54,renovated_55,renovated_56,renovated_57,renovated_58,renovated_59,renovated_60,renovated_61,renovated_62,renovated_63,renovated_64,renovated_65,renovated_66,renovated_67,renovated_68,renovated_69,renovated_70,renovated_71,renovated_72,renovated_73,renovated_74,renovated_75,renovated_76,renovated_77,renovated_78,renovated_79,renovated_80,renovated_81,renovated_82,renovated_83,renovated_84,renovated_85,renovated_86,renovated_87,renovated_88,renovated_89,renovated_90,renovated_91,renovated_92,renovated_93,renovated_94,renovated_95,renovated_96,renovated_97,renovated_98,renovated_99,renovated_100,renovated_101,renovated_102,renovated_103,renovated_104,renovated_105,renovated_106,renovated_107,renovated_108,renovated_109,renovated_110,renovated_111,renovated_112,renovated_113,renovated_114,renovated_115,renovated_116,renovated_117,renovated_118,renovated_119,renovated_120
0,0,0,0,0,0,0,1,0,0,0,0,0,

In [24]:
holdout_cont = holdout.drop(['bedrooms','floors','waterfront','view','condition','yr_built','zipcode','month_sold','years_since_renovation'],axis=1)
scaler = StandardScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(holdout_cont[['sqft_living','sqft_lot','sqft_basement','lat','long']]),columns=['sqft_living','sqft_lot','sqft_basement','lat','long'])

In [41]:
new_holdout = pd.concat([scaled_df, dummie_data], axis=1)
new_holdout.rename(columns={'floors_3.0':'floors_3'}, inplace=True)
new_holdout.rename(columns={'floors_3.5':'floors_3_5'}, inplace=True)
new_holdout.rename(columns={'floors_1.5':'floors_1_5'}, inplace=True)
new_holdout.rename(columns={'floors_2.0':'floors_2'}, inplace=True)
new_holdout.rename(columns={'floors_2.5':'floors_2_5'}, inplace=True)
new_holdout.rename(columns={'bed_2.0':'bed_2'}, inplace=True)
new_holdout.rename(columns={'bed_3.0':'bed_3'}, inplace=True)
new_holdout.rename(columns={'bed_4.0':'bed_4'}, inplace=True)
new_holdout.rename(columns={'bed_5.0':'bed_5'}, inplace=True)
new_holdout.rename(columns={'bed_6.0':'bed_6'}, inplace=True)
new_holdout.rename(columns={'bed_7.0':'bed_7'}, inplace=True)
new_holdout.rename(columns={'bed_8.0':'bed_8'}, inplace=True)
new_holdout.rename(columns={'bed_9.0':'bed_9'}, inplace=True)
new_holdout.rename(columns={'bed_10.0':'bed_10'}, inplace=True)
new_holdout.rename(columns={'bed_11.0':'bed_11'}, inplace=True)

edit_holdout = new_holdout.drop(['zip_98199'], axis = 1)
# new_holdout.head()

In [42]:
prediction = model.predict(edit_holdout)

In [51]:
prediction.mean()

718223.2141717086

In [52]:
edit_holdout.mean()

sqft_living      1.147975e-16
sqft_lot        -1.586620e-16
sqft_basement    1.532483e-15
lat             -2.714721e-14
long            -1.015436e-13
                     ...     
renovated_116    9.252834e-04
renovated_117    1.850567e-03
renovated_118    6.939625e-04
renovated_119    2.313208e-04
renovated_120    1.850567e-03
Length: 338, dtype: float64